In [1]:
import os
import asyncio
import nest_asyncio

In [2]:
%pwd

'c:\\Users\\etrou\\OneDrive\\Desktop\\SE489GroupProjectGit\\group_project_se489\\notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\etrou\\OneDrive\\Desktop\\SE489GroupProjectGit\\group_project_se489'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from se489_group_project.constants import *
from se489_group_project.utility.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.data_storage])


    
    def get_training_config(self) -> ModelTrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = ModelTrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
import webbrowser
import subprocess
import torch
from torch.profiler import profile, record_function, ProfilerActivity


In [9]:
class Training:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config

    
    async def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        
    async def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    async def stop_profiler_if_running(self):
        """Stops the profiler if it is currently running."""
        try:
            tf.profiler.experimental.stop()
        except tf.errors.UnimplementedError:
            print("Profiler was not running.")
        except tf.errors.FailedPreconditionError:
            print("No profiler is running.")
        except Exception as e:
            print(f"An unexpected error occurred while stopping the profiler: {e}")

    
    async def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        log_dir = "se489_group_project/visualizations/tensorboard_logs"
        
        os.makedirs(log_dir, exist_ok=True)
       
        # Stop any running profiler instances before starting a new one
        await self.stop_profiler_if_running()

        #TensorFlow Profiling 
        tf.profiler.experimental.start(log_dir)

         # Set up TensorBoard callback
        tensorboard_callback = tf.keras.callbacks.TensorBoard(
            log_dir=log_dir,
            histogram_freq=1,
            profile_batch='500,520'  # Profile batches 500 to 520
        )
        try:
            self.model.fit(
                self.train_generator,
                epochs=self.config.params_epochs,
                steps_per_epoch=self.steps_per_epoch,
                validation_steps=self.validation_steps,
                validation_data=self.valid_generator,
                callbacks=[tensorboard_callback]
            )
        finally:
        
            tf.profiler.experimental.stop()
        
       
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )
        
    

In [10]:
import cProfile
import webbrowser
#async added to main function to improve performance metrics
async def main():
        logs = "se489_group_project/visualizations/tensorboard_logs"
        
        subprocess.Popen(["tensorboard", "--logdir", logs, "--host", "0.0.0.0", "--port", "6006"])
        time.sleep(5)
        webbrowser.open("http://localhost:6006/")

        log_dir = os.path.join(os.getcwd(), "se489_group_project", "visualizations")
        file = os.path.join(log_dir, 'cprofile_stats_model_training.prof')
        config = ConfigurationManager()
        training_config = config.get_training_config()
        training = Training(config=training_config)
        profiler = cProfile.Profile()
        profiler.enable()
        await training.get_base_model()
        await training.train_valid_generator()
        await training.train()
        profiler.disable()
        profiler.dump_stats(file)
        profile_file_full_path = os.path.abspath(file)
        
        #Automatically open snakeviz to visualize the profiling results
        try:
            subprocess.Popen(["snakeviz", profile_file_full_path])
        except FileNotFoundError:
            print("snakeviz is not installed or not found in the system path.")
    

if __name__ == "__main__":
    nest_asyncio.apply()
    asyncio.run(main())



[2024-05-30 16:28:17,448: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-30 16:28:17,452: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-30 16:28:17,453: INFO: common: created directory at: data]
[2024-05-30 16:28:17,454: INFO: common: created directory at: data\training]
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
An unexpected error occurred while stopping the profiler: Cannot export profiling results. No profiler is running.
[2024-05-30 16:28:18,123: ERROR: callbacks: Failed to start profiler: Another profiler is running.]
23/23 [==============================] - 33s 1s/step - loss: 0.4279 - accuracy: 0.8034 - val_loss: 0.5747 - val_accuracy: 0.8000
